# Домашние задания четвертого урока.

**1. Какую из представленных функций часто используют для приведения типов?**

* DISTINCT
* **CAST**
* FROM
* EXTRACT

# Далее в заданиях используются данные с платформы Airbnb в Берлине ✈️.

Имеются следующие таблицы:

1. **listings** – информация о жилье, включая полные описания, характеристики и средние оценки в отзывах; поскольку столбцов очень много, нужные перечислены в текстах самих задач

2. **calendar_summary** – информация о доступности и цене того или иного жилья по дням

* listing_id – идентификатор жилья (объявления)
* date – дата
* available – доступность жилья в данный день (t/f)
* price – цена за ночь

3. **reviews** – отзывы

* listing_id –  идентификатор жилья  
* id – id отзыва
* date – дата отзыва
* reviewer_id – id ревьюера (автора отзыва)
* reviewer_name – имя автора
* comments – сам отзыв


**2. На Airbnb есть две основные группы пользователей:**

* Хозяева (хосты, hosts) – сдают жилье
* Путешественники (travelers) – снимают

**Начнем с анализа характеристик хозяев в таблице listings!**

**Пользователи, сдающие квартиры на Airbnb, зарегистрировались в разное время. Кто-то – очень давно, а кто-то совсем недавно. Давайте проверим, в какой месяц и год зарегистрировалось наибольшее количество новых хостов. В качестве ответа введите дату следующего формата: 2010-12**

* host_id – идентификатор хозяина
* host_since – дата регистрации как хост

Note: Сам хост может встретиться в таблице несколько раз, если сдает несколько помещений, поэтому не забудьте оставить уникальные значения host_id. Также обратите внимание на тип данных в host_since, возможно вам пригодится toStartOfMonth() для извлечения части даты и toDateOrNull() для обработки пустых значений.
```sql
SELECT 
    COUNT(DISTINCT host_id) AS Hosts_per_month,
    toStartOfMonth(toDateOrNull(host_since)) as month     
FROM
    listings     
GROUP BY
    month    
ORDER BY
    Hosts_per_month DESC    
LIMIT 1
```

<br> NumHosts лучше имя чем Hosts_per_month

**3. Посмотрим на среднюю частоту ответа среди хозяев (f) и суперхозяев (t).**

**Значения частоты ответа хранятся как строки и включают значок %, который необходимо заменить на пустоту (''). После этого приведите столбец к нужному типу данных с помощью toInt32OrNull() и посчитайте среднюю частоту отклика в разбивке по тому, является ли хост суперхозяином или нет. В качестве ответа укажите наибольшее среднее.**

* host_response_rate – частота ответа
* host_is_superhost – является ли суперхозяином

Важный момент: у каждого host_id есть только одно уникальное значение частоты ответа, а также одна единая отметка суперхозяина. Чтобы посчитать показатели честно, нужно использовать подзапрос и взять уникальные комбинации с помощью DISTINCT.

* AVG нуллы не берет в расчет

```sql
SELECT
    host_is_superhost,    
    AVG(toInt32OrNull(replaceAll(host_response_rate, '%', ''))) as avg_rate  
FROM
    (SELECT     
        DISTINCT         
            host_id,            
            host_response_rate,            
            host_is_superhost           
    FROM listings     
    ) as name
WHERE 
    host_is_superhost != ''    
GROUP BY
    host_is_superhost
ORDER BY 
    avg_rate DESC
LIMIT 1
```
* на sql-ex подзапрос обязательно именовать
* изначально я не сделал WHERE host_is_superhost != '' т.к. не имею данных, чтобы проверять наличие

**4. Сгруппируйте данные из listings по хозяевам (host_id) и посчитайте, какую цену за ночь в среднем каждый из них устанавливает (у одного хоста может быть несколько объявлений). Идентификаторы сдаваемого жилья объедините в отдельный массив. Таблицу отсортируйте по убыванию средней цены и убыванию host_id (в таком порядке). В качестве ответа укажите первый массив в результирующей таблице, состоящий более чем из двух id. Например:**

2343477,6340,32324

* host_id – идентификатор хозяина
* id – идентификатор жилья
* price – цена за ночь в конкретном месте
```sql
SELECT 
    host_id,   
    groupArray(id) as ids,     
    AVG(toFloat64OrNull(replaceRegexpAll(price, '[$,]', ''))) AS AvgPricePerHost  
FROM
    listings   
GROUP BY
    host_id   
ORDER BY
    AvgPricePerHost DESC,
    host_id DESC
LIMIT 10
```

* HAVING len(ids)>=2 не нужно - ORDER BY сортирует по длине массива


**5. Немного усложним предыдущую задачу, и посчитаем разницу между максимальной и минимальной установленной ценой у каждого хозяина. В качестве ответа укажите идентификатор хоста, у которого разница оказалась наибольшей.**

* host_id – идентификатор хозяина
* id – идентификатор жилья
* price – цена за ночь в конкретном месте
```sql
SELECT 
    host_id, 
    MAX(price)-MIN(price) as price_diff
FROM 
    (SELECT host_id, toFloat64OrNull(replaceRegexpAll(price, '[$,]', '')) as price
    FROM listings)
GROUP BY 
    host_id
ORDER BY 
    price_diff DESC
LIMIT 1
```

* более громоздкий вариант, но также более информативный   
```sql
    MIN(toFloat64OrNull(replaceRegexpAll(price, '[$,]', ''))) as MinPricePerHost,   
    MAX(toFloat64OrNull(replaceRegexpAll(price, '[$,]', ''))) as MaxPricePerHost,  
    MaxPricePerHost - MinPricePerHost AS price_diff 
```

**6. Теперь сгруппируйте данные по типу жилья и выведите средние значения цены за ночь, размера депозита и цены уборки. Обратите внимание на тип данных, наличие значка $ и запятых в больших суммах. Для какого типа жилья среднее значение залога наибольшее?**

* room_type – тип сдаваемого жилья 
* price – цена за ночь
* security_deposit – залог за сохранность имущества
* cleaning_fee – плата за уборку
```sql
SELECT
    room_type,   
    AVG(toFloat32OrNull(replaceRegexpAll(price, '[$,]', ''))) as AvgPrice,   
    AVG(toFloat32OrNull(replaceRegexpAll(security_deposit, '[$,]', ''))) as AvgSecurityDeposit,   
    AVG(toFloat32OrNull(replaceRegexpAll(cleaning_fee, '[$,]', ''))) as AvgCleanFee   
FROM
    listings    
GROUP BY
    room_type    
ORDER BY
    AvgSecurityDeposit DESC
```



**7. Отлично! В каких частях города средняя стоимость за ночь является наиболее низкой?**

**Сгруппируйте данные по neighbourhood_cleansed и посчитайте среднюю цену за ночь в каждом районе. В качестве ответа введите название места, где средняя стоимость за ночь ниже всего.**

* price – цена за ночь
* neighbourhood_cleansed – район/округ города
```sql
SELECT
    neighbourhood_cleansed,    
    AVG(toFloat32OrNull(replaceRegexpAll(price, '[$,]', ''))) as AvgPrice
FROM
    listings    
GROUP BY
    neighbourhood_cleansed     
ORDER BY
    AvgPrice    
LIMIT 10
```


**8. В каких районах Берлина средняя площадь жилья, которое сдаётся целиком, является наибольшей? Отсортируйте по среднему и выберите топ-3.**

* neighbourhood_cleansed – район
* square_feet – площадь
* room_type – тип (нужный – 'Entire home/apt')
```sql
SELECT
    neighbourhood_cleansed,    
    AVG(toFloat32OrNull(square_feet)) as AvgArea    
FROM
    listings    
WHERE
    room_type = 'Entire home/apt'    
GROUP BY
    neighbourhood_cleansed    
ORDER BY
    AvgArea DESC   
LIMIT 10
```


**9. Напоследок давайте посмотрим, какая из представленных комнат расположена ближе всего к центру города. В качестве ответа укажите id объявления.**

* id – идентификатор жилья (объявления)
* room_type – тип жилья ('Private room')
* latitude – широта
* longitude – долгота
* 52.5200 с.ш., 13.4050 в.д – координаты центра Берлина
```sql
SELECT 
    id,   
    toFloat64OrNull(latitude) as latitude,    
    toFloat64OrNull(longitude) as longitude,   
    geoDistance(13.4050, 52.5200, longitude, latitude) as dist   
FROM
    listings    
WHERE 
    room_type = 'Private room'  
ORDER BY
    dist   
```

* в лекции не было replaceRegexpAll + re2 syntax и подзапросов

* не практиковали:
* coalesce

* DECIMAL и проблемы float 
* Nullable - это просто название типа а не функция
* interval/datetime..